In [2]:
import json
with open('gpt3-output.json') as f:
    data = json.load(f)

In [29]:
# Allows interactive plotting
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt, random

# This is my first time using iPy, so this is a bit clunky...
def series(i):
    for sent in [x[i] for x in data.values()]:
        print(sent + "\n")
    return()

interact(
    series, 
    i = widgets.IntSlider(
        value=0,
        min=0,
        max=len(data['naive'])-1,
        step=1,
        description='Sentence:',
        orientation='horizontal'
    )
)

# data=[data], 
# good_deletion=(-20,20,0.5), 
# good_trivial_insertion=(-20,20,0.5), 
# good_insertion=(-20,20,0.5), 
# good_paraphrase=(-20,20,0.5), 
# good_syntax=(-20,20,0.5), 
# grammar_error=(-20,20,0.5), 
# content_error=(-20,20,0.5),
# size_calculation=['linear', 'log', 'square', 'none'],
# average=[True, False],
# user=['all'] + sorted(list(set([sent['user'] for sent in data]))),
# display_distribution=[True, False]

interactive(children=(IntSlider(value=0, description='Sentence:', max=27), Output()), _dom_classes=('widget-in…

<function __main__.series(i)>

`text-davinci-002` vs `text-davinci-003`

In [20]:
import json
import csv
import os

In [55]:
def edit_dist(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

def avg(lst, prec=2):
    if len(lst) == 0:
        return 0
    return round(sum(lst) / len(lst), prec)

In [59]:
def load_different_models(batch_num_range):
    out = []
    for batch_num in batch_num_range:
        path = os.getcwd()
        files = [f'{path}\\{x}' for x in os.listdir(path) if 'json' in x and 'few' in x]

        d2 = [f for f in files if 'davinci-002' in f and f'batch-{batch_num}' in f]
        d3 = [f for f in files if 'davinci-002' not in f and f'batch-{batch_num}' in f]

        with open(d2[0]) as f:
            data_d2 = json.load(f)

        with open(d3[0]) as f:
            data_d3 = json.load(f)

        filename = f'batch-{batch_num}.csv'
        # Load data
        with open(filename, encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)
            data = [row for row in reader]
        # Will only use the sentences
        orig = [str(x[0]) for x in data]
        
        for i in range(len(orig)):
            out.append({
                'batch': batch_num,
                'original': orig[i],
                'text-davinci-002': data_d2[i],
                'text-davinci-003': data_d3[i],
                'models-edit-dist': edit_dist(data_d2[i], data_d3[i]),
                'orig-edit-dist': edit_dist(orig[i], data_d3[i]),
                'models-length-change': len(data_d2[i]) - len(data_d3[i]),
            })

    return out

In [60]:
data = load_different_models([1, 2])

In [61]:
# Some example sentences
data[:2]

[{'batch': 1,
  'original': "The architecture of Winchester College is a diverse set of architectural styles, reflecting the multiple periods of building from the college's foundation in 1382, through additions in the medieval and Early Modern periods, to a major expansion of accommodation in the Victorian era and then further extensions at the turn of the 20th century and more recently.",
  'text-davinci-002': "The architecture of Winchester College has a diverse set of styles that reflects the multiple periods of building from the college's foundation in 1382. This includes additions in the medieval and Early Modern periods, to a major expansion of accommodation in the Victorian era and then further extensions at the turn of the 20th century and more recently.",
  'text-davinci-003': 'Winchester College was built in several architectural styles over its long history, beginning in 1382. Additions and extensions were made in the medieval and Early Modern periods, and then more addition

In [62]:
avg([x['models-length-change'] for x in data])

15.92

In [63]:
avg([x['models-edit-dist'] for x in data])

106.4

In [64]:
avg([x['orig-edit-dist'] for x in data])

113.65